In [2]:
import numpy as np 
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')

In [3]:
df = pd.read_csv('Fake_Real_Data.csv')
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [4]:
df['label'].value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [5]:
df['label'] = df['label'].map({'Fake' : 0, 'Real' : 1})

In [6]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0
1,U.S. conservative leader optimistic of common ...,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",1
3,Court Forces Ohio To Allow Millions Of Illega...,0
4,Democrats say Trump agrees to work on immigrat...,1


In [11]:
df['vector'] = df['Text'].apply(lambda text : nlp(text).vector)

In [13]:
df.head()

,Text,label,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,1,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",1,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,0,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,1,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."


In [16]:
df['vector']

0       [-0.6759837, 1.4263071, -2.318466, -0.451093, ...
1       [-1.8355803, 1.3101058, -2.4919677, 1.0268308,...
2       [-1.9851209, 0.14389805, -2.4221718, 0.9133005...
3       [-2.7812982, -0.16120885, -1.609772, 1.3624227...
4       [-2.2010763, 0.9961637, -2.4088492, 1.128273, ...
                              ...                        
9895    [-1.6682401, 0.78006977, -2.2337353, -0.159771...
9896    [-1.9297235, 0.8007302, -1.8990824, 0.42668718...
9897    [-1.5289013, 1.0250993, -1.9861357, 0.4278564,...
9898    [-1.3928099, 0.7792715, -2.2072845, 0.13192406...
9899    [-1.5856593, -0.31769615, -0.9302766, 0.330447...
Name: vector, Length: 9900, dtype: object

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['vector'].values, df['label'], random_state=42, test_size=0.2)

In [18]:
x_train

array([array([-2.0152547 ,  0.98641217, -2.3584037 ,  0.7322799 ,  4.107265  ,
               0.22396342,  0.6253973 ,  3.5500612 , -0.3170374 , -1.8648638 ,
               5.48375   ,  1.7104373 , -2.89802   ,  0.74601495, -0.29321775,
               2.350302  ,  1.2645686 , -0.06555936, -1.020395  , -0.9757464 ,
               1.2710329 , -0.8918504 ,  0.03230966, -0.19836223, -0.11627487,
              -1.1124969 , -1.7526532 , -0.5143565 , -0.88816553,  0.6904575 ,
               0.63850135,  0.09136863, -0.65409815, -2.1196    , -1.8729937 ,
              -0.9783039 , -0.09043716,  0.0499923 ,  1.3933578 ,  0.3026516 ,
               0.86849004,  0.11710268,  0.08253215, -0.04134313, -1.6167058 ,
               0.97282106,  1.4764651 , -1.9817255 , -0.51629865,  1.8665777 ,
              -1.3808936 ,  1.5644169 , -0.42669407, -4.5888352 , -0.8152755 ,
               1.0388777 ,  0.4114647 ,  0.7135278 ,  0.6071856 , -0.52639794,
               0.32114387, -0.6968656 ,  0.4408755 ,

In [19]:
## For x we need to give a 2d array for training purpose so for that we need to use np.stack
x_train2d = np.stack(x_train)
x_test2d = np.stack(x_test)

In [23]:
## First scale the vector
from sklearn.preprocessing import MinMaxScaler
minMax = MinMaxScaler()
minMax.fit(x_train2d)
x_train2d = minMax.transform(x_train2d)
x_test2d = minMax.transform(x_test2d)

In [24]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train2d, y_train)
nbPred = nb.predict(x_test2d)

In [30]:
from sklearn.metrics import classification_report, confusion_matrix
print("The classification report is: \n", classification_report(y_test, nbPred))
print("The confusion matrix is: \n", confusion_matrix(y_test, nbPred))

The classification report is: 
               precision    recall  f1-score   support

           0       0.95      0.94      0.95       973
           1       0.94      0.96      0.95      1007

    accuracy                           0.95      1980
   macro avg       0.95      0.95      0.95      1980
weighted avg       0.95      0.95      0.95      1980

The confusion matrix is: 
 [[913  60]
 [ 44 963]]


In [27]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors= 5 , metric='euclidean')
knn.fit(x_train2d, y_train)
knnPred = knn.predict(x_test2d)

In [ ]:
from sklearn.metrics import accuracy_score

In [34]:
print("The classification report is: \n", classification_report(y_test, knnPred))
print("The confusion matrix is: \n", confusion_matrix(y_test, knnPred))
print("The accuracy is : ", accuracy_score(y_test, knnPred), "\n")

The classification report is: 
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       973
           1       0.99      1.00      0.99      1007

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

The confusion matrix is: 
 [[ 958   15]
 [   2 1005]]
The accuracy is :  0.9914141414141414 



In [33]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train2d, y_train)
rfPred =  rf.predict(x_test2d)

print("The accuracy is : ", accuracy_score(y_test, rfPred), "\n")
print("The classification report is: \n", classification_report(y_test, rfPred))
print("The confusion matrix is: \n", confusion_matrix(y_test, rfPred))

The accuracy is :  0.9934343434343434 

The classification report is: 
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       973
           1       0.99      1.00      0.99      1007

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

The confusion matrix is: 
 [[ 962   11]
 [   2 1005]]
